This notebook takes raw data from each source specified below in their respective sections, cleans and preprocessed it into one Master DataFrame with all featuresThis notebook is broken into the following sections: <br>

* Imports and installs
* Functions
* Download and preprocess data
* Combine data into one master dataframe

# Imports and Installs

In [14]:
# Admin
import requests
from google.colab import files
from datetime import datetime, timedelta
from tqdm import tqdm
from tqdm.notebook import tqdm
import time

import ast
import pandas as pd
import numpy as np
import yfinance as yf
from functools import reduce

from sklearn.model_selection import TimeSeriesSplit
from statsmodels.api import OLS, GLS, add_constant
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import r2_score, make_scorer, mean_squared_error
import statsmodels.api as sm
from statsmodels.stats.diagnostic import het_breuschpagan
from scipy.stats import pearsonr
import scipy.stats as stats


# Sentiment
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline
import nltk
from nltk.tokenize import sent_tokenize
from scipy.stats import mannwhitneyu


# Visualizations
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [15]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# Functions

In [16]:
def download_stock_prices(ticker_symbol, start):
  """
  Downloads historical adjusted closing prices for a given stock ticker.

  Parameters:
  - ticker_symbol: The stock ticker symbol as a string.

  Returns:
  - prices_df: A DataFrame containing the adjusted closing prices.
  """
  start = start
  today_date = datetime.now().strftime('%Y-%m-%d')
  end = '2024-03-06'

  # Download historical data
  price_data = yf.download(ticker_symbol, start=start, end=end)

  # Extract adjusted closing prices
  adj_close_prices = price_data['Adj Close']

  # Create a DataFrame with the adjusted closing prices
  prices_df = pd.DataFrame(adj_close_prices)

  return prices_df

In [17]:
def interpolate_daily_values(column):
  # Create a dataframe
  daily_df = pd.DataFrame(columns = ['Adj Close', 'Daily Avg'])

  for i in range(len(column)):
    # Get the index value of the input column
    a = column.index[i]

    # Find the index position in the prices dataframe that is closest to the index value in the revenue dataframe
    idx_pos = prices.index.searchsorted(a)

    # Get the row number of the index value in the prices dataframe
    row_index = idx_pos + 1

    # Calculate average daily value by dividing the value of the input column by the number of rows (days) in the prices dataframe
    average_daily_value = column.iloc[i] / (row_index-len(daily_df))

    # Create a smaller dataframe with data for only quarter i that will be concatenated to the daily_df
    sub_df = pd.DataFrame(columns=['Adj Close', 'Daily Avg'])
    sub_df['Adj Close'] = prices.iloc[len(daily_df):row_index]
    sub_df['Daily Avg'] = average_daily_value


    daily_df = pd.concat([daily_df, sub_df])

  daily_df.drop(columns = ['Adj Close'])

  return daily_df

In [18]:
def download_and_clean_transcripts():
  transcripts_df = pd.DataFrame()
  year_list = [2020, 2021, 2022, 2023,2024]
  api_key = ''

  for year in year_list:
    url = f'https://financialmodelingprep.com/api/v4/batch_earning_call_transcript/MSFT?year={year}&apikey={api_key}'
    data = requests.get(url).json()
    if data and isinstance(data, list):
      df = pd.DataFrame(data)
      transcripts_df = transcripts_df.append(df, ignore_index=True)

  transcripts_df = transcripts_df.drop(columns=['symbol', 'quarter', 'year'])
  transcripts_df['date'] = pd.to_datetime(transcripts_df['date']).dt.date
  transcripts_df['content'] = transcripts_df['content'].str.replace('\n', '', regex=False)
  transcripts_df = transcripts_df.rename(columns={'date': 'Date', 'content': 'Text'})
  transcripts_df = transcripts_df.sort_values(by='Date', ascending=True)

  return transcripts_df

In [19]:
# Download the finBERT model,
# This will output a sentiment label and score for a dataframe with 'Date' and 'Text' columns

# Initialize the tokenizer and model for ProsusAI/finBERT
tokenizer = AutoTokenizer.from_pretrained('ProsusAI/finBERT')
model = AutoModelForSequenceClassification.from_pretrained('ProsusAI/finBERT')

# Create a sentiment analysis pipeline using the finBERT model
finbert_sentiment = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)

# Define a function to apply sentiment analysis to each row
def get_sentiment(row):

    results = finbert_sentiment(row['Text'], truncation=True, max_length=512)

    return pd.Series([row['Text'], results[0]['label'], results[0]['score']])

In [20]:
def calculate_sentiment_scores(transcripts_df):

  ticker_avg_sentiment = []

  for i in tqdm(range(len(transcripts_df))):
    transcript = transcripts_df['Text'].iloc[i]
    sentences = sent_tokenize(transcript)

    # skip calls that have less than 4 tokens
    if len(sentences) < 4:
      continue

    # Trim first two and last 2 items from sentence list
    sentences = sentences[2:-2]
    # Create a dataframe to calculate the score for each item in the sentence list.
    sentences_df = pd.DataFrame(sentences, columns=['Text'])
    sentiment_results = sentences_df.apply(get_sentiment, axis=1)
    sentiment_results.columns = ['Sentence', 'Label', 'Score']
    # Remove neutral scores
    sentiment_results = sentiment_results[sentiment_results['Label'] != 'neutral']
    # Remove sentences with low sentiment
    rows_to_drop = int(len(sentiment_results) * 0.3)
    sentiment_results = sentiment_results.sort_values(by = 'Score', ascending = True).iloc[rows_to_drop:]
    # Convert negative sentiment scores
    sentiment_results['Score'] = sentiment_results.apply(lambda row: -row['Score'] if row['Label'] == 'negative' else row['Score'], axis=1)
    # Calculate average sentiment score for the call
    avg_sentiment = sum(sentiment_results['Score'])/len(sentiment_results)
    ticker_avg_sentiment.append(avg_sentiment)

  transcripts_df['Sentiment'] = ticker_avg_sentiment

  return transcripts_df

In [21]:
def mean_return_over_n_days(df, N):
  """
  Calculates the price returns over the previous N days for the 'Adj Close' column in a DataFrame.

  Parameters:
  - df: A pandas DataFrame containing the 'Adj Close' column with prices.
  - N: The number of days over which to calculate the returns.

  Returns:
  - The mean value of the 'Return Over N Days' column.
  """
  # Calculate the percentage change over the previous N days and store it in a new column
  df['Return Over N Days'] = df['Adj Close'].pct_change(periods=N)

  # Calculate the mean value of the 'Return Over N Days' column
  mean_return_over_n_days = round(df['Return Over N Days'].mean(),4)

  return mean_return_over_n_days

In [22]:
def shifted_returns_df(df, N):
  """
  Creates a new DataFrame with price returns over N days and shifted returns.

  Parameters:
  - df: A pandas DataFrame containing an 'Adj Close' column with prices.
  - N: The number of days over which to calculate the returns.

  Returns:
  - A new DataFrame 'shifted_returns' with columns 'Adj Close', 'Return Over N Days',
    and 'Shifted Return Over N Days'.
  """
  # Initialize the new DataFrame
  shifted_returns = pd.DataFrame()
  shifted_returns['Adj Close'] = df['Adj Close']

  # Calculate the percentage change over the previous N days for 'Return Over N Days'
  shifted_returns['Return Over N Days'] = df['Adj Close'].pct_change(periods=N)

  # Shift the 'Return Over N Days' by N days to create 'Shifted Return Over N Days'
  shifted_returns['Shifted Return Over N Days'] = shifted_returns['Return Over N Days'].shift(-N)

  return shifted_returns

# Download Data

## Stock Price Data

Data obtained from Yahoo! Finance using yfinance package

In [23]:
msft_df = download_stock_prices('MSFT', '2020-01-01')
msft_df.head()

[*********************100%%**********************]  1 of 1 completed


,Adj Close
Date,
2020-01-02,154.493820
2020-01-03,152.570129
2020-01-06,152.964462
2020-01-07,151.569794
2020-01-08,153.984039


In [24]:
fig = px.line(msft_df, y='Adj Close', title='MSFT Adjusted Close Price')

fig.update_xaxes(title_text='Date')
fig.update_yaxes(title_text='Adjusted Close Price ($)')
fig.update_traces(line=dict(color="#00274C"))

fig.show()

## Implied Volatility

Implied volatility is a measure used to estimate the degree of future price variability of a stock, based on the prices of its options. It reflects investors' predictions about the asset's potential movement but doesn't indicate the direction of the movement.<br>Data obtained from Nasdaq Data Link (formerly Quandl) using an API key.

In [25]:
implied_volatility = pd.read_csv('msft_implied_volatility_raw.csv')
implied_volatility['date'] = pd.to_datetime(implied_volatility['date'])
implied_volatility.rename(columns={'date': 'Date'}, inplace=True)
implied_volatility.set_index('Date', inplace=True)
implied_volatility = implied_volatility.drop(columns = 'ticker')
implied_volatility = implied_volatility.sort_values(by='Date', ascending=True)

In [26]:
merged_df = msft_df.merge(implied_volatility, on='Date', how='inner')
fig = px.scatter(merged_df, x='ivmean30', y='Adj Close', title='30 Day Implied Volatility vs MSFT Adjusted Close Price', trendline="lowess")
fig.update_xaxes(title_text='30 Day Average Implied Volatility')
fig.update_yaxes(title_text='MSFT Adjusted Close Price ($)')

fig.show()

## CBOE Volatility

The CBOE Volatility Index (VIX), often called the 'fear gauge' is a popular measure that represents the stock market's expectation of volatility over the next 30 days, based on options prices of the S&P 500 index. While the VIX measures the market's overall volatility expectations, implied volatility refers to the expected volatility of a specific security or asset, in this case MSFT

In [27]:
# Set starting and ending dates
start_date = '2020-01-01'
end_date = '2024-03-05'

# List of ticker symbols
vix_tickers = ['^VIX9D', '^VIX', '^VIX3M', '^VIX6M']

# Create an empty DataFrame with a full date index
vix_df = pd.DataFrame(index=pd.date_range(start_date, end_date))

# Loop through each ticker symbol
for vix_ticker in vix_tickers:
    try:
        # Download the data for the current ticker
        price = yf.download(vix_ticker, start=start_date, end=end_date)['Adj Close']

        # Add the downloaded data to the DataFrame using ffill to fill forward missing values
        vix_df[vix_ticker] = price.reindex(vix_df.index, method='ffill').fillna(method='ffill')
    except Exception as e:
        print(f"Error downloading data for {vix_ticker}: {e}")


vix_df = round(vix_df, 4)

vix_df = vix_df.rename_axis('Date')

common_index = msft_df.index.intersection(vix_df.index)

vix_df = vix_df.loc[common_index]

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


## Stock Grade

The stock rating provided by hedge funds, investment firms, and analysts. Data sourced from Financial Modeling Prep using an API Key

In [28]:
grade_df = pd.read_csv('msft_grade_raw.csv')
grade_df = grade_df.drop(columns = ['symbol', 'gradingCompany'])

grade_to_category = {
    'Sell': 1, 'Underperform': 1, 'Underweight': 1,
    'Hold': 2, 'Neutral': 2, 'Market perform': 2, 'Perform': 2, 'Equal-Weight': 2, 'Sector Perform': 2, 'Equal-weight': 2,
    'Overweight': 3, 'Buy': 3, 'Outperform': 3,
    'Strong Buy': 4, 'Long-term buy': 4, 'Long-Term Buy': 4
}

# Function to map grade to category
def map_grade_to_category(grade):
  return grade_to_category.get(grade, None)


def calculate_grade_change(row):

  if pd.isnull(row['previousGrade']) or row['previousGrade'] == '':
    return map_grade_to_category(row['newGrade'])
  else:
    prev_cat = map_grade_to_category(row['previousGrade'])
    new_cat = map_grade_to_category(row['newGrade'])
    if prev_cat is not None and new_cat is not None:
      return new_cat - prev_cat
    else:
      return None

# Apply the function row-wise
grade_df['Grade Change'] = grade_df.apply(calculate_grade_change, axis=1)

def map_new_grade_to_category(new_grade):
  return grade_to_category.get(new_grade, None)
# Apply the function to the 'newGrade' column to create the new 'Grade' column
grade_df['Grade'] = grade_df['newGrade'].apply(map_new_grade_to_category)

grade_df.rename(columns={'date': 'Date'}, inplace=True)

grade_df['Date'] = pd.to_datetime(grade_df['Date'])

grade_df = grade_df.groupby('Date')[['Grade Change', 'Grade']].mean().reset_index()

grade_df = grade_df[grade_df['Date'] > pd.Timestamp('2019-12-31')]

grade_df = grade_df.set_index('Date').dropna()


In [29]:
expanded_grade_df = grade_df.reindex(msft_df.index, method='ffill')
merged_df = msft_df.merge(expanded_grade_df, left_index=True, right_index=True, how='left')
grade_df = merged_df.drop('Adj Close', axis = 1)
grade_df.iloc[:4] = grade_df.iloc[4]

## Analyst Recommendation

This is an analyst's opinion on the future performance of MSFT stock, expressed as strong sell, strong buy, buy, hold, or sell based on analysis of the Microsoft's fundamentals, industry position, and market conditions.

In [30]:
df = pd.read_csv('msft_analyst_recommendations_raw.csv')

df['analystRecommendations'] = (
    "Buy: " + df['analystRatingsbuy'].astype(str) +
    ", Hold: " + df['analystRatingsHold'].astype(str) +
    ", Sell: " + df['analystRatingsSell'].astype(str) +
    ", Strong Sell: " + df['analystRatingsStrongSell'].astype(str) +
    ", Strong Buy: " + df['analystRatingsStrongBuy'].astype(str)
)

weights = {
    'analystRatingsStrongSell': -2,
    'analystRatingsSell': -1,
    'analystRatingsHold': 0,
    'analystRatingsbuy': 1,
    'analystRatingsStrongBuy': 2
}

# Calculating weighted score
for column, weight in weights.items():
    df[column + '_weighted'] = df[column] * weight

# Sum the weighted scores into a single score
df['weightedScore'] = df[[col + '_weighted' for col in weights]].sum(axis=1)



In [31]:
recommendation_score = (
    df[['date', 'weightedScore']]  # Select relevant columns
    .rename(columns={'date': 'Date'})  # Rename 'date' to 'Date'
    .assign(Date=lambda x: pd.to_datetime(x['Date']))  # Convert 'Date' to datetime
    .query("Date > '2019-12-31'")  # Filter dates after 2019-12-31
    .sort_values('Date', ascending=True)  # Sort by 'Date'
    .set_index('Date')
    .dropna()
)

expanded_rec_score = recommendation_score.reindex(msft_df.index, method='ffill')
merged_df = msft_df.merge(expanded_rec_score, left_index=True, right_index=True, how='left')
recommendation_score = merged_df.drop('Adj Close', axis = 1)

## Trading Volume

The total number of stocks traded on a daily basis, indicating the overall activity and liquidity for MSFT in the market. <br>Data sourced from Yahoo! Finance

In [32]:
ticker_symbol = 'MSFT'
start_date = '2020-01-01'
end_date = '2024-03-06'
volume = yf.download(ticker_symbol, start=start_date, end=end_date)
volume = volume.drop(columns = ['Open','High','Low','Close','Adj Close'])

[*********************100%%**********************]  1 of 1 completed


## Income Statements (Quarterly)

Income statements for each quarter. The following code resamples this to daily amounts

In [33]:
quarterly_income_statements = pd.read_csv('msft_q_income_statements_raw.csv')
daily_is = (quarterly_income_statements
            .drop(columns=['weightedAverageShsOut', 'weightedAverageShsOutDil',
                           'grossProfitRatio', 'ebitdaratio', 'operatingIncomeRatio',
                           'incomeBeforeTaxRatio', 'netIncomeRatio','fillingDate'])
            .rename(columns={'date': 'Date'})
            .assign(Date=lambda x: pd.to_datetime(x['Date']))
            .sort_values(by='Date', ascending=True)
            .set_index('Date')
            .dropna())

In [34]:
prices = msft_df
ful_daily_is = pd.DataFrame(index=daily_is.index)

for column_name in daily_is.columns:
  # Use the function on each column
  interpolated_df = interpolate_daily_values(daily_is[column_name])

  # Drop the 'Adj Close' column from the output
  if 'Adj Close' in interpolated_df.columns:
    interpolated_df.drop(columns=['Adj Close'], inplace=True)

  # Change the column title of 'Daily Avg' to the name of the column from the daily_is DataFrame
  interpolated_df.rename(columns={'Daily Avg': column_name}, inplace=True)

  # Concatenate this output to ful_daily_is
  ful_daily_is = pd.concat([ful_daily_is, interpolated_df], axis=1)

daily_is = ful_daily_is
daily_is = daily_is.dropna()

In [35]:
combined_index = daily_is.index.union(msft_df.index)
daily_is_reindexed = daily_is.reindex(combined_index, method='ffill')

The income statement data ends at December, 2023. We will use the average growth rate from the 4th Quarter of 2023 to extrapolate values for January - March 2024 using income statement growth data from Financial Modeling Prep.

In [36]:
data = pd.read_csv('msft_is_growth.csv')
data = pd.DataFrame(data)

In [37]:
data = data.drop(columns= ['date', 'symbol', 'calendarYear', 'period',
                  'growthGrossProfitRatio', 'growthEBITDARatio',
                  'growthOperatingIncomeRatio', 'growthIncomeBeforeTaxRatio',
                  'growthNetIncomeRatio', 'growthWeightedAverageShsOut',
                  'growthWeightedAverageShsOutDil'])


In [38]:
# Some columns in the income statement don't have a corresponding column in the income statement growth table.
# For this we will use the average growth rate across all columns for that quarter.
average = data.iloc[0].mean()
average

0.10087824294736843

In [39]:
growth_to_column_mapping = {
    'growthRevenue': 'revenue',
    'growthCostOfRevenue': 'costOfRevenue',
    'growthGrossProfit': 'grossProfit',
    'growthResearchAndDevelopmentExpenses': 'researchAndDevelopmentExpenses',
    'growthGeneralAndAdministrativeExpenses': 'generalAndAdministrativeExpenses',
    'growthSellingAndMarketingExpenses': 'sellingAndMarketingExpenses',
    'growthOtherExpenses': 'otherExpenses',
    'growthOperatingExpenses': 'operatingExpenses',
    'growthCostAndExpenses': 'costAndExpenses',
    'growthInterestExpense': 'interestExpense',
    'growthDepreciationAndAmortization': 'depreciationAndAmortization',
    'growthEBITDA': 'ebitda',
    'growthOperatingIncome': 'operatingIncome',
    'growthTotalOtherIncomeExpensesNet': 'totalOtherIncomeExpensesNet',
    'growthIncomeBeforeTax': 'incomeBeforeTax',
    'growthIncomeTaxExpense': 'incomeTaxExpense',
    'growthNetIncome': 'netIncome',
    'growthEPS': 'eps',
    'growthEPSDiluted': 'epsdiluted'
}

growth_rates_df = pd.DataFrame(index=daily_is.index, columns=daily_is.columns)

# Populate growth_rates_df with growth rates using the mapping
for growth_key, column_name in growth_to_column_mapping.items():
  if column_name in growth_rates_df.columns:
    # Assign the value from data[0] to the correct column in growth_rates_df
    growth_rates_df[column_name] = data.iloc[0].get(growth_key, None)
growth_rates_df = growth_rates_df.iloc[-1]
growth_rates_df.fillna(average, inplace=True)

In [40]:
daily_is_complete = pd.read_csv('msft_daily_income_statements.csv', parse_dates=['Date']).set_index('Date')

Income statement line items like those relating to shares and ratios can't be interpolated using the method above, but should be forward filled.

In [41]:
shares_ratios = pd.DataFrame(data = quarterly_income_statements[['weightedAverageShsOut', 'weightedAverageShsOutDil',
                           'grossProfitRatio', 'ebitdaratio', 'operatingIncomeRatio',
                           'incomeBeforeTaxRatio', 'netIncomeRatio','date']] ).dropna().rename(columns={'date': 'Date'}).set_index('Date')

growth_rates = shares_ratios.pct_change()

# Calculate the average of these growth rates for each column
average_growth_rates = growth_rates.mean()

shares_ratios.index = pd.to_datetime(shares_ratios.index)
shares_ratios = shares_ratios.reindex(msft_df.index)
shares_ratios = shares_ratios.ffill()

In [42]:
# Since the data starts on March, 2020, the previous values need to be backfilled.
# Calculate the adjusted values for backfilling using the values at index 61
values_at_62nd_row = shares_ratios.iloc[61]
adjusted_values = values_at_62nd_row / (1 - average_growth_rates)

# Create a DataFrame with adjusted values repeated for the first 61 rows
adjusted_df = pd.DataFrame([adjusted_values] * 61, index=shares_ratios.index[:61])

shares_ratios.update(adjusted_df)

shares_ratios.ffill(inplace=True)

## Balance Sheet

Provides a snapshot of a company's financial position at a specific point in time. Data obtained from Financial Modeling Prep.

In [43]:
qbs = pd.read_csv('msft_q_balance_sheet.csv')

In [44]:
dbs = (qbs.assign(Date=pd.to_datetime(qbs['Date']))
         .set_index('Date')
         .reindex(msft_df.index)
         .interpolate(method='linear'))

In [45]:
starting_values = qbs.iloc[0]
ending_values = dbs.iloc[61]

# Calculate the number of points to interpolate, including the start and end
num_points = 62

# Step 2: Generate interpolated values for the first 61 rows
for column in dbs.columns:
  # Calculate slope (m)
  slope = (ending_values[column] - starting_values[column]) / (num_points - 1)

  # Calculate y-intercept (b), using y = mx + b, where x = 0 for the start
  intercept = starting_values[column]

  # Generate interpolated values for each point
  interpolated_values = [intercept + slope * x for x in range(num_points)]

  # Fill in the NaN values in the first 61 rows
  dbs[column].iloc[:62] = interpolated_values

# Ensure interpolation didn't unintentionally modify values beyond the first 62 rows
dbs.iloc[62:] = dbs.iloc[62:].ffill()

## Cash Flow Statement

A summary of a company's cash inflows and outflows over a period of time.

In [46]:
qcf = pd.read_csv('msft_q_cash_flow.csv')
qcf = (qcf.assign(Date=pd.to_datetime(qcf['Date']))
         .set_index('Date'))
qcf = qcf.iloc[1:]

In [47]:
prices = msft_df
ful_daily_cf = pd.DataFrame(index=prices.index)

for column_name in qcf.columns:

  interpolated_df = interpolate_daily_values(qcf[column_name])

  if 'Adj Close' in interpolated_df.columns:
    interpolated_df.drop(columns=['Adj Close'], inplace=True)

  interpolated_df.rename(columns={'Daily Avg': column_name}, inplace=True)

  ful_daily_cf = pd.concat([ful_daily_cf, interpolated_df], axis=1)

dcf = ful_daily_cf
dcf = dcf.dropna()

In [48]:
daily_cf = pd.read_csv('msft_daily_cash_flow.csv', parse_dates=['Date']).set_index('Date')

## Earnings Surprise

The difference between actual earnings per share and earnings expected by analysts prior to earnings release. Data obtained from Financial Modeling Prep.

In [49]:
surprise = pd.read_csv('msft_earnings_surprise.csv')
surprise = (surprise.assign(Date=pd.to_datetime(surprise['Date']))
         .set_index('Date')
         .reindex(msft_df.index)
         .fillna(0))

## Earnings Call Transcripts

Transcripts from quarterly earnings calls. Data obtained from AlphaVantage.

In [51]:
msft_transcripts = pd.read_csv('msft_transcripts_raw.csv')
msft_transcripts = pd.DataFrame(msft_transcripts)

In [52]:
msft_transcripts = msft_transcripts.drop(columns=['symbol', 'quarter', 'year'])
msft_transcripts['date'] = pd.to_datetime(msft_transcripts['date']).dt.date
msft_transcripts['content'] = msft_transcripts['content'].str.replace('\n', '', regex=False)
msft_transcripts = msft_transcripts.rename(columns={'date': 'Date', 'content': 'Text'})
msft_transcripts = msft_transcripts.sort_values(by='Date', ascending=True)

In [53]:
msft_transcripts.head()

,Date,Text
2,2006-01-27,Operator: Good afternoon and welcome to the Mi...
1,2006-04-28,"Operator: Good afternoon, and welcome to the M..."
0,2006-07-21,Operator: Good afternoon and welcome to the Mi...
6,2006-10-27,"Operator: Good afternoon, and welcome to the M..."
5,2007-01-26,Operator: Welcome to the Microsoft 2007 fiscal...


In [1]:
# this is a long runtime code block. The output from this is in the file msft_transcripts_with_sentiment.csv
msft_avg_sentiment = []

for i in tqdm(range(len(msft_transcripts))):
  transcript = msft_transcripts['Text'].iloc[i]
  sentences = sent_tokenize(transcript)
  # Trim first two and last 2 items from sentence list
  sentences = sentences[2:-2]
  # Create a dataframe to calculate the score for each item in the sentence list.
  sentences_df = pd.DataFrame(sentences, columns=['Text'])
  sentiment_results = sentences_df.apply(get_sentiment, axis=1)
  sentiment_results.columns = ['Sentence', 'Label', 'Score']
  # Remove neutral scores
  sentiment_results = sentiment_results[sentiment_results['Label'] != 'neutral']
  # Remove sentences with low sentiment
  rows_to_drop = int(len(sentiment_results) * 0.3)
  sentiment_results = sentiment_results.sort_values(by = 'Score', ascending = True).iloc[rows_to_drop:]
  # Convert negative sentiment scores
  sentiment_results['Score'] = sentiment_results.apply(lambda row: -row['Score'] if row['Label'] == 'negative' else row['Score'], axis=1)
  # Calculate average sentiment score for the call
  avg_sentiment = sum(sentiment_results['Score'])/len(sentiment_results)
  msft_avg_sentiment.append(avg_sentiment)

msft_transcripts['Sentiment'] = msft_avg_sentiment

In [ ]:
# Save DataFrame to a CSV file
csv_filename = 'msft_transcripts_with_sentiment.csv'
msft_transcripts.to_csv(csv_filename, index=False)

# Download the CSV file to your local machine
files.download(csv_filename)

In [61]:
transcripts = pd.read_csv('msft_transcripts_with_sentiment.csv')
transcripts = pd.DataFrame(transcripts[['Date','Sentiment']]).drop_duplicates().set_index('Date').reindex(msft_df.index).fillna(0)

In [64]:
transcripts.tail()

,Sentiment
Date,
2024-02-28,0.0
2024-02-29,0.0
2024-03-01,0.0
2024-03-04,0.0
2024-03-05,0.0


## Congress Trades

In [65]:
congress = (pd.read_csv('msft_congress.csv')
              .drop(columns=['Range', 'House', 'Party', 'last_modified', 'Ticker', 'ReportDate', 'Unnamed: 0', 'Representative'])
              .replace({'Transaction': {'Sale (Partial)': 'Sale', 'Sale (Full)': 'Sale'}})
              .rename(columns={'TransactionDate': 'Date'})
              .assign(Date=lambda df: pd.to_datetime(df['Date'])))

congress['congress_net_trade'] = np.where(congress['Transaction'] == 'Purchase',
                                          congress['Amount'],
                                          -congress['Amount'])

congress = (congress.drop(columns=['Transaction', 'Amount'])
                             .groupby('Date')['congress_net_trade']
                             .sum())
congress = pd.DataFrame(congress)

In [66]:
congress = congress.reindex(msft_df.index).fillna(0)

Is there an increase in the price of MSFT in the days following a purchase transaction?

In [67]:
msft_congress = pd.read_csv('msft_congress.csv')
msft_congress_df = pd.DataFrame(msft_congress)

# MSFT Purchase Transactions
msft_purchase_transaction = msft_congress_df[msft_congress_df['Transaction'] == 'Purchase'].copy()
msft_purchase_transaction = msft_purchase_transaction.drop(columns = ['Representative','Transaction', 'Unnamed: 0'])
msft_purchase_transaction = msft_purchase_transaction.sort_values(by='TransactionDate', ascending=True)
msft_purchase_transaction['Amount'] = msft_purchase_transaction['Amount'].astype(float)
two_lowest_values = msft_purchase_transaction['Amount'].nsmallest(2).unique()
msft_purchase_transaction = msft_purchase_transaction[~msft_purchase_transaction['Amount'].isin(two_lowest_values)]
msft_purchase_transaction['TransactionDate'] = pd.to_datetime(msft_purchase_transaction['TransactionDate'])
msft_purchase_transaction = msft_purchase_transaction.sort_values(by = 'TransactionDate')
msft_purchase_transaction.head()

,ReportDate,TransactionDate,Ticker,Range,House,Amount,Party,last_modified
780,8/29/2014,2013-07-19,MSFT,"$15,001 - $50,000",Representatives,15001.0,R,11/16/2023
779,8/29/2014,2013-09-15,MSFT,"$15,001 - $50,000",Representatives,15001.0,R,11/16/2023
761,8/12/2014,2014-07-08,MSFT,"$15,001 - $50,000",Representatives,15001.0,R,11/16/2023
742,7/28/2015,2015-06-23,MSFT,"$15,001 - $50,000",Representatives,15001.0,D,11/16/2023
741,7/28/2015,2015-06-26,MSFT,"$50,001 - $100,000",Representatives,50001.0,D,11/16/2023


In [68]:
msft_mean_return = []
msft_post_transaction_return = []

for i in range(250): #250 trading days in one year

  # mean return for N number of days
  msft_mean_return.append(mean_return_over_n_days(msft_df,i))

  # return for N number of days following a congress purchase
  n_day_shifted_return = shifted_returns_df(msft_df,i)
  merged_df = pd.merge_asof(msft_purchase_transaction, n_day_shifted_return.reset_index(), left_on='TransactionDate', right_on='Date', direction='forward')
  msft_post_transaction_return.append(round(np.mean(merged_df['Shifted Return Over N Days']),4))

In [69]:
msft_comparison = pd.DataFrame({
    'Number of Trading Days': range(1, 251),
    'mean_return': msft_mean_return,
    'post_transaction_return': msft_post_transaction_return,
})
msft_comparison['average_excess_return'] = msft_comparison['post_transaction_return'] - msft_comparison['mean_return']

In [70]:
fig = px.line(msft_comparison, x='Number of Trading Days', y='average_excess_return', color_discrete_sequence=['#00274C'],
              title='Average Excess Return Over Number of Trading Days')

fig.update_xaxes(title_text='Number of Trading Days Post Purchase')
fig.update_yaxes(title_text='Average Excess Return')
fig.add_hline(y=0, line_color='#FFCB05')

fig.show()

The chart above shows the difference in return for MSFT stock over the days after a Congress member makes a purchase compared to the return over a "normal" time period of the same length. Around 63 day post-trade period, we can see this excess return coming in to effect. This can potentially be seen as a leading indicator that on average MSFT, once a Congress member has purchased MSFT stock, we will see higher returns than average over the 2 mont to 1 year period post-purchase.

In [71]:
msft_df = msft_df.drop(columns = 'Return Over N Days')

## Ratings

Analyst rating of MSFT. Data obtained from Financial Modeling Prep

In [72]:
ratings = pd.read_csv('msft_ratings.csv').assign(Date=lambda df: pd.to_datetime(df['Date'])).set_index('Date')
ratings = ratings.drop(columns = ['ratingRecommendation', 'ratingDetailsDCFRecommendation', 'ratingDetailsROERecommendation', 'ratingDetailsROARecommendation', 'ratingDetailsDERecommendation', 'ratingDetailsPERecommendation', 'ratingDetailsPBRecommendation'])

In [73]:
ratings = ratings.reindex(msft_df.index).fillna(method='ffill')

## Social Media Sentiment

counts from Nov 18, 2021 to Jan 21, 2024 <br> Sentiment scores from Feb 20, 2022 to Jan 21, 2024 for StockTwits and Apr 4, 2023 for Twitter

Data obtained from Financial Modeling Prep

In [74]:
# this function wa used to fetch social media data from Financial Modeling Prep
def fetch_social_media_data(url, api_key):
  all_data = []
  page = 0
  continue_fetching = True

  while continue_fetching:
    paginated_url = f"{url}&page={page}"
    response = requests.get(paginated_url.format(api_key=api_key))
    data = response.json()

    # Convert the current page of data to a DataFrame
    df = pd.DataFrame(data)

    if not df.empty:
      # Check the last value in the 'date' column
      last_date = pd.to_datetime(df['date'].iloc[-1])

      # If the last date is on or after January 1, 2020, add the data and pull the next page
      if last_date >= pd.Timestamp('2020-01-01'):
        all_data.append(df)
      # If the last date is before December 31, 2023, stop fetching
      elif last_date < pd.Timestamp('2020-01-01'):
        continue_fetching = False

      page += 1
    else:
      break

  # Concatenate all the pages of data into a single DataFrame
  if all_data:
    social_sentiment = pd.concat(all_data, ignore_index=True)
    # Filter the DataFrame for dates
    social_sentiment['date'] = pd.to_datetime(social_sentiment['date'])
    social_sentiment = social_sentiment[social_sentiment['date'].dt.year > 2019]
    return social_sentiment
  else:
    return pd.DataFrame()

In [76]:
social_sentiment = pd.read_csv('msft_social_sentiment.csv')
social_sentiment = pd.DataFrame(social_sentiment)
social_sentiment['Date'] = pd.to_datetime(social_sentiment['Date'])
social_sentiment['Date'] = social_sentiment['Date'].dt.strftime('%Y-%m-%d')

In [77]:
columns_to_convert = ['stocktwitsSentiment', 'twitterSentiment']
for col in columns_to_convert:
  social_sentiment[col] = pd.to_numeric(social_sentiment[col], errors='coerce')
  social_sentiment[col] = social_sentiment[col].fillna(0)

aggregations = {
    'stocktwitsPosts': 'sum', 'twitterPosts': 'sum',
    'stocktwitsComments': 'sum', 'twitterComments': 'sum',
    'stocktwitsLikes': 'sum', 'twitterLikes': 'sum',
    'stocktwitsImpressions': 'sum', 'twitterImpressions': 'sum',
    'stocktwitsSentiment': 'mean', 'twitterSentiment': 'mean'
}

# Group by 'Date', then apply the aggregation operations
social_sentiment = social_sentiment.groupby('Date').agg(aggregations).reset_index()

In [78]:
social_sentiment = social_sentiment.set_index('Date')
social_sentiment.index = pd.to_datetime(social_sentiment.index)
social_sentiment = social_sentiment.reindex(msft_df.index).fillna(value = 0)

## Reddit

Reddit data available from March 9, 2022. <br>

Data was downloaded from the subreddits r/wallstreetbets and r/stocks for the queries "MSFT" and "Microsoft" using the following 4 cells repeated 4 times.

The Reddit instance created to pull this data was based on the teachings from the Reddit Lab assignment of SIADS 682 Scial Media Analytics:<br>Lang, Erik. "Reddit API Lab - Create." SIADS682 Social Media Analytics, submitted 12 March, 2024, University of Michigan. Unpublished course assignment.

In [81]:
wallstreetbets_msft = pd.read_csv('reddit_wallstreetbets_msft_data.csv')
wallstreetbets_msft = pd.DataFrame(wallstreetbets_msft)
wallstreetbets_microsoft = pd.read_csv('reddit_wallstreetbets_microsoft_data.csv')
wallstreetbets_microsoft = pd.DataFrame(wallstreetbets_microsoft)
stocks_msft = pd.read_csv('reddit_stocks_msft_data.csv')
stocks_msft = pd.DataFrame(stocks_msft)
stocks_microsoft = pd.read_csv('reddit_stocks_microsoft_data.csv')
stocks_microsoft = pd.DataFrame(stocks_microsoft)

In [83]:
reddit = pd.concat([
    wallstreetbets_msft[['title', 'text', 'created_utc']],
    wallstreetbets_microsoft[['title', 'text', 'created_utc']],
    stocks_msft[['title', 'text', 'created_utc']],
    stocks_microsoft[['title', 'text', 'created_utc']]
], ignore_index=True)

In [84]:
# Filter out comments
reddit =reddit[~reddit['title'].str.startswith('Comment on')].copy()
reddit['created_utc'] = pd.to_datetime(reddit['created_utc']).dt.date
reddit = reddit.rename(columns={'created_utc': 'Date'})
reddit['full_text'] = reddit.apply(lambda row: row['title'] + '. ' + row['text'] if pd.notna(row['text']) else row['title'], axis=1)
reddit = reddit.drop(['title','text'], axis=1)

reddit = reddit.rename(columns={'full_text': 'Text'})
reddit = reddit.sort_values(by='Date', ascending=True)

In [85]:
# # Apply the function to the dataframe and create new columns for 'Label' and 'Score'
# reddit[['Label', 'Score']] = reddit.apply(get_sentiment, axis=1)

reddit_scores = pd.read_csv('reddit_scores.csv')
reddit_scores = pd.DataFrame(reddit_scores).drop(columns=['Unnamed: 0'])

In [86]:
# Calculate a sentiment score that combines the label and the score

def combine_label_score(label, score):
    if label == 'positive':
        return score  # Positive score as is
    elif label == 'negative':
        return -score  # Negative score as a negative value
    else:
        return score/10 # Neutral sentiment divide score by 10

reddit_scores['redditSentiment'] = reddit_scores.apply(lambda row: combine_label_score(row['Label'], row['Score']), axis=1)
reddit_scores = reddit_scores.groupby('Date')['redditSentiment'].mean().reset_index()
reddit_scores = reddit_scores.set_index('Date')
reddit_scores = reddit_scores.reindex(msft_df.index).fillna(value = 0)

## News Sentiment

News data starting from August 16, 2022.

Data obtained from Financial Modeling Prep

In [87]:
# function to download news data from Financial Modeling Prep

def fetch_news_data(api_key):
  all_news_data = []
  page = 0

  while True:
      url = f'https://financialmodelingprep.com/api/v4/stock-news-sentiments-rss-feed?page={page}&apikey={api_key}'
      response = requests.get(url)
      data = response.json()

      if not data:
        break  # Stop if there's no data

      df = pd.DataFrame(data)

      if 'symbol' not in df.columns:
        break  # Stop if the expected column is not in the dataframe

      msft_news = df[df['symbol'] == 'MSFT']

      if not msft_news.empty:
        msft_news['publishedDate'] = pd.to_datetime(msft_news['publishedDate'], utc=True)
        all_news_data.append(msft_news)

      page += 1

  if all_news_data:
    all_news_data = pd.concat(all_news_data, ignore_index=True)
    # Filter for articles after 2019 after all data is collected
    all_news_data = all_news_data[(all_news_data['publishedDate'] >= '2020-01-01') &
                                  (all_news_data['publishedDate'] < '2024-03-06')]
    return all_news_data
  else:
    return pd.DataFrame()

In [88]:
news = (pd.read_csv('msft_news_data.csv')
        .drop(columns=['title', 'site', 'text', 'sentiment'])
        .rename(columns={'sentimentScore': 'news_sentimentScore'}))

news['Date'] = pd.to_datetime(news['Date']).dt.strftime('%Y-%m-%d')

news = news.groupby('Date')['news_sentimentScore'].mean().reset_index()
news = news.set_index('Date')
news.index = pd.to_datetime(news.index)
news = news.reindex(msft_df.index).fillna(value = 0)

## AlphaVantage News

More news articles about MSFT from different sources.

Data obtained from AlphaVantage using an API key.

In [90]:
relevant_data_all_columns = pd.read_csv('av_news_data.csv')
relevant_data_all_columns['ticker_sentiment'] = relevant_data_all_columns['ticker_sentiment'].apply(ast.literal_eval)

In [91]:
relevant_data = pd.DataFrame()
relevant_data['Date'] = relevant_data_all_columns['time_published']
# Trim the contents of the 'Date' column to keep only the first 8 characters
relevant_data['Date'] = relevant_data['Date'].str[:8]
# Convert 'Date' column to datetime type with the format YYYY-MM-DD
relevant_data['Date'] = pd.to_datetime(relevant_data['Date'], format='%Y%m%d')

In [92]:
# Initialize an empty list to store the sentiment scores
sentiment_scores = []

# Iterate over the 'ticker_sentiment' column
for sentiment_list in relevant_data_all_columns['ticker_sentiment']:
  msft_sentiment_score = None

  # Iterate over the list of dictionaries in the 'ticker_sentiment' column
  for sentiment_dict in sentiment_list:
    # Check if the 'ticker' value starts with 'MSFT'
    if 'ticker' in sentiment_dict and sentiment_dict['ticker'].startswith('MSFT'):
      # Get the 'ticker_sentiment_score' value from the dictionary
      msft_sentiment_score = sentiment_dict.get('ticker_sentiment_score')
      break

  # Append the sentiment score to the list
  sentiment_scores.append(msft_sentiment_score)

# Convert list of strings to list of floats using map() function
sentiment_scores_float = list(map(float, sentiment_scores))

# Add the 'AV Sentiment Score' column to the relevant_data DataFrame
relevant_data['AV Sentiment Score'] = sentiment_scores_float

In [93]:
relevant_data['text'] = relevant_data_all_columns['title'] + '. ' + relevant_data_all_columns['summary']

def preprocess_text(text):

  return text.lower().strip()

relevant_data['text'] = relevant_data['text'].apply(preprocess_text)

In [54]:
# this cell has a long runtime. The output from this code is in the file converted_scores.csv
# # Use the finBERT model to predict sentiment for each text
sentiment_results = finbert_sentiment(relevant_data['text'].tolist())

# # Extract sentiment labels and scores (adjust based on model output format)
relevant_data['sentiment'] = [result['label'] for result in sentiment_results]
relevant_data['finBERT sentiment_score'] = [result['score'] for result in sentiment_results]

In [95]:
converted_scores_with_text = pd.read_csv('converted_scores.csv')
# Convert 'Date' column to DateTime
converted_scores_with_text['Date'] = pd.to_datetime(converted_scores_with_text['Date'])
# Set 'Date' column as index
converted_scores_with_text.set_index('Date', inplace=True)
converted_scores = converted_scores_with_text.drop(columns=['text','sentiment','finBERT sentiment_score'])
converted_scores = converted_scores.rename(columns={'converted_score': 'finbert_news_score'})
av_news = converted_scores
av_news = av_news.groupby('Date').mean()
av_news = av_news.reindex(msft_df.index).fillna(value = 0)

## Dividends

Dividend declarartion and payment data for MSFT.

Data obtained from Financial Modeling Prep

In [55]:
df = pd.read_csv('msft_dividends.csv')

In [56]:
div_payment = (df[['adjDividend', 'paymentDate']]
               .rename(columns={'adjDividend': 'Dividend', 'paymentDate': 'Date'})
               .assign(Date=lambda x: pd.to_datetime(x['Date']))
               .set_index('Date')
               .reindex(msft_df.index, fill_value=0))

In [57]:
div_declar = (df[['adjDividend', 'declarationDate']]
              .drop_duplicates('declarationDate')
               .rename(columns={'adjDividend': 'Dividend', 'declarationDate': 'Date'})
               .assign(Date=lambda x: pd.to_datetime(x['Date']))
               .set_index('Date')
               .reindex(msft_df.index, fill_value=0))

In [60]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=msft_df.index, y=msft_df['Adj Close'], mode='lines',
                         line=dict(color='#00274C'),
                         name='Adj Closing Price'))

# Dummy trace for dividend declarations legend entry
fig.add_trace(go.Scatter(x=[None], y=[None], mode='lines',
                         line=dict(color='#FFCB05', width=1),
                         name='Dividend Declaration Date'))

# Add vertical lines for dividend declaration dates
for date in div_declar_filtered.index:
    fig.add_vline(x=date, line_width=1, line_color='#FFCB05')

# Update layout with titles and axis labels
fig.update_layout(
    title="MSFT Adjusted Closing Price with Dividend Declaration Dates",
    xaxis_title="Date",
    yaxis_title="Adjusted Close Price ($)",
    xaxis=dict(showgrid=False),
    yaxis=dict(showgrid=False),
)

fig.show()

## Ratios

Key ratios for MSFT

Data obtained from Financial Modeling Prep

In [100]:
ratios = pd.read_csv('msft_ratios.csv')
ratios = pd.DataFrame(ratios)
ratios['Date'] = pd.to_datetime(ratios['Date'])

ratios = ratios.set_index('Date')
first_row = ratios.iloc[0]
first_row = first_row.to_frame().T
ratios = ratios.reindex(msft_df.index)
ratios = pd.concat([first_row, ratios], axis=0)
ratios = ratios.interpolate(method='linear', axis=0)
ratios = ratios.iloc[1:]

## Key Metrics

Key metrics for MSFT.

Data obtained from Financial Modeling Prep

In [101]:
metrics = pd.read_csv('msft_metrics.csv')
metrics = pd.DataFrame(metrics)
metrics['Date'] = pd.to_datetime(metrics['Date'])

metrics = metrics.set_index('Date')
metrics = metrics.drop(columns = ['marketCap','netIncomePerShare'])
first_row = metrics.iloc[0]
first_row = first_row.to_frame().T
metrics = metrics.reindex(msft_df.index)
metrics = pd.concat([first_row, metrics], axis=0)
metrics = metrics.interpolate(method='linear', axis=0)
metrics = metrics.iloc[1:]

# Master Dataframe

Merge features to one master dataframe

In [102]:
# List of all DataFrames to be merged
dfs = [msft_df, vix_df, implied_volatility, grade_df, recommendation_score,
       volume, daily_is_complete, shares_ratios, dbs, daily_cf, surprise,
       transcripts, congress, ratings, social_sentiment, news, div_payment,
       div_declar, ratios, metrics, reddit_scores, av_news]

# Use reduce to apply pd.merge in a chain across all DataFrames in the list
master_df = reduce(lambda left, right: pd.merge(left, right, left_index=True, right_index=True, how='inner'), dfs)

master_df['Shifted Adj Close'] = master_df['Adj Close'].shift(-1)

last_features_row = master_df.drop(columns=['Adj Close','Shifted Adj Close']).iloc[-1].values.reshape(1, -1)

master_df = master_df.dropna()
master_df.head()

,Adj Close,^VIX9D,^VIX,^VIX3M,^VIX6M,hv10,hv20,hv30,hv60,hv90,...,averageInventory,daysSalesOutstanding,daysPayablesOutstanding,daysOfInventoryOnHand,roe,capexPerShare,redditSentiment,AV Sentiment Score,finbert_news_score,Shifted Adj Close
Date,,,,,,,,,,,,,,,,,,,,,
2020-01-02,154.493820,10.60,12.47,15.19,16.58,0.1334,0.1156,0.1043,0.0988,0.1347,...,2.214613e+09,57.384287,64.356106,13.279729,0.105603,-0.465652,0.0,0.0,0.0,152.570114
2020-01-03,152.570114,12.78,14.02,16.01,17.16,0.1627,0.1315,0.1170,0.1056,0.1363,...,2.206726e+09,57.399854,64.544062,13.283037,0.105412,-0.466142,0.0,0.0,0.0,152.964462
2020-01-06,152.964462,12.42,13.85,15.92,17.08,0.1627,0.1287,0.1170,0.1053,0.1349,...,2.198839e+09,57.415421,64.732018,13.286346,0.105220,-0.466631,0.0,0.0,0.0,151.569794
2020-01-07,151.569794,12.34,13.79,15.86,17.02,0.1660,0.1352,0.1240,0.1084,0.1343,...,2.190952e+09,57.430987,64.919974,13.289654,0.105028,-0.467121,0.0,0.0,0.0,153.984070
2020-01-08,153.984070,12.22,13.45,15.40,16.65,0.1947,0.1455,0.1283,0.1135,0.1330,...,2.183065e+09,57.446554,65.107929,13.292963,0.104836,-0.467611,0.0,0.0,0.0,155.907791


In [103]:
# Save DataFrame to a CSV file
csv_filename = 'master_df.csv'
master_df.to_csv(csv_filename, index=True)

# Download the CSV file
files.download(csv_filename)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>